# Import necessary libraries



In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


# Load datasets

In [2]:
customers = pd.read_csv('Customers.csv')
transactions = pd.read_csv('Transactions.csv')



# Aggregate transaction data for each customer


In [3]:
customer_transactions = transactions.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum'
}).reset_index()


# Merge with customer data

In [4]:
customer_data = pd.merge(customers, customer_transactions, on='CustomerID', how='left').fillna(0)



# Normalize numeric columns for similarity calculation


In [5]:
scaler = MinMaxScaler()
numeric_cols = ['TotalValue', 'Quantity']
customer_data[numeric_cols] = scaler.fit_transform(customer_data[numeric_cols])



# Compute cosine similarity


In [6]:
similarity_matrix = cosine_similarity(customer_data[numeric_cols])
similarity_df = pd.DataFrame(similarity_matrix, index=customer_data['CustomerID'], columns=customer_data['CustomerID'])


# Recommend top 3 similar customers for the first 20 customers


In [7]:
lookalike_results = {}
for customer in customer_data['CustomerID'][:20]:
    similar_customers = similarity_df[customer].sort_values(ascending=False)[1:4]  # Exclude itself
    lookalike_results[customer] = list(zip(similar_customers.index, similar_customers.values))


# Save the results to a CSV file


In [8]:
lookalike_df = pd.DataFrame({
    'CustomerID': lookalike_results.keys(),
    'Lookalikes': [v for v in lookalike_results.values()]
})
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Lookalike recommendations saved to Lookalike.csv.")

Lookalike recommendations saved to Lookalike.csv.
